In [ ]:
pip install implicit

     |████████████████████████████████| 1.1 MB 10.9 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406620 sha256=9579c5eae13bbe161f7177b2828e16096d194df5d592c8e95bdb98b44ffd0088
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [ ]:
#импортируем библиотеки
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle 
import implicit
from scipy import sparse as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#датасет транзакций
transactions = pd.read_csv("//content/drive/MyDrive/skillbox-recommender-system.zip (Unzipped Files)/transactions.csv")

In [ ]:
#отсортируем датасет
transactions.sort_values('order_id',inplace=True)

In [ ]:
#train_test_split 70% на 30%
train = transactions[:18485651]
test = transactions[18485651:]

In [ ]:
#создадим множество user_id для кодировки ID
user_id_set = set(transactions['user_id'])
#создадим множество product_id для кодировки ID
product_id_set = set(transactions['product_id'])

In [ ]:
#словари ключ словаря содержит значение user_id/product_id значение ключа содержит index для кодировки
dict_user_id = {}
dict_product_id = {}

In [ ]:
#словарь для раскодировки
encode_user_id = {}
encode_product_id = {}

In [ ]:
#кодируем user_id
for idx,values in tqdm(enumerate(user_id_set)):
    dict_user_id.update({values:idx})

In [ ]:
#кодируем product_id
for idx,values in tqdm(enumerate(product_id_set)):
    dict_product_id.update({values:idx})

In [ ]:
#раскодируем user_id
for i in tqdm(dict_user_id.keys()):
  encode_user_id.update({dict_user_id.get(i):i})

In [ ]:
#раскодируем product_id
for i in tqdm(dict_product_id.keys()):
  encode_product_id.update({dict_product_id.get(i):i})

In [ ]:
#инициализируем нулевую матрицу количество пользователей на продукты
#test_matrix = np.zeros((transactions['user_id'].nunique(),transactions['product_id'].nunique()),dtype='uint8')
#train_matrix = np.zeros((transactions['user_id'].nunique(),transactions['product_id'].nunique()),dtype='uint8')
#Матрица количества покупок
# emb_pur_count = np.zeros((transactions['user_id'].nunique(),transactions['product_id'].nunique()),dtype='uint8')


In [ ]:
# #заполняем test матрицу 
# for row in tqdm(test.iterrows()):
#     test_matrix[dict_user_id.get(int(row[1]['user_id']))][dict_product_id.get(int(row[1]['product_id']))] = 1

In [ ]:
# #заполняем train матрицу 
# for row in tqdm(train.iterrows()):
#     train_matrix[dict_user_id.get(int(row[1]['user_id']))][dict_product_id.get(int(row[1]['product_id']))] = 1

In [ ]:
#заполним свою матрицу количества покупок
# for row in tqdm(transactions.iterrows()):
#   emb_pur_count[dict_user_id.get(int(row[1]['user_id']))][dict_product_id.get(int(row[1]['product_id']))] += 1

In [ ]:
f = open('/content/drive/MyDrive/skillbox-recommender-system.zip (Unzipped Files)/test_matrix.pkl','rb')
test_matrix = pickle.load(f)
f.close()

In [ ]:
f = open('/content/drive/MyDrive/skillbox-recommender-system.zip (Unzipped Files)/train_matrix.pkl','rb')
train_matrix = pickle.load(f)
f.close()

In [ ]:
f = open('/content/drive/MyDrive/skillbox-recommender-system.zip (Unzipped Files)/emb_pur_count.pkl','rb')
emb_pur_count = pickle.load(f)
f.close()

Получим топ покупаемых товаров далее идем в цикле по матрице и получаем список покупок в датасете train и считаем метрику

In [ ]:
top_group = train.groupby('product_id',as_index=False).count()
top_group.sort_values('user_id',ascending=False,inplace=True)

In [ ]:
top_products = top_group['product_id'][:30]

In [ ]:
top = []
for i in top_products:
    top.append(dict_product_id.get(i))
    

In [ ]:
def average_precision(actual, recommended, k=10):
    ap_sum = 0
    hits = 0
    for i in range(k):
        product_id = recommended[i] if i < len(recommended) else None
        if product_id in actual:
            hits += 1
            ap_sum += hits / (i + 1)
    return ap_sum / k


def normalized_average_precision(actual, recommended, k=10):
    #actual = set(actual)
    if len(actual) == 0:
        return 0.0

    ap = average_precision(actual, recommended, k=k)
    ap_ideal = average_precision(actual, list(actual)[:k], k=k)
    return ap / ap_ideal

Посчитаем метрику, подставим топ 10 товаров всем пользоваталям ( будем считать это минимум метрики)

In [ ]:
#идем в цикле по матрице, получаем список покупок ( gt_items ) и считаем метрику
scores = []
for i in tqdm(test_matrix):
    gt_items = np.nonzero(i)[0]
    #print(gt_items)
    ap = normalized_average_precision(gt_items, top)
    scores.append(ap)
print(np.mean(scores))


0.1283308363945578


In [ ]:
# #матрицу для всего датасета
# full_matrix = np.zeros((transactions['user_id'].nunique(),transactions['product_id'].nunique()),dtype='uint8')
# for row in tqdm(transactions.iterrows()):
#     full_matrix[dict_user_id.get(int(row[1]['user_id']))][dict_product_id.get(int(row[1]['product_id']))] = 1

In [ ]:
#функция которая принимает список транзацкий по пользователю и user_id пользователя, подготавливает данные к подаче модели.
#используем количество товаров как фича, пробовал разные вариации, оставляем самую эффективную
#функция возвращает обьект coo_matrix

def make_coo_row(transaction_history,d_user_id):
    idx = [] #id товара
    values = [] #количество товара
    enc_user_id = encode_user_id.get(d_user_id) #id пользователя
    items = [] # id товара
    #бежим в цикле по транзакциям и добавляем product_id в items
    for trans in transaction_history:
        items.append(trans)
    #записываем количество продуктов
    n_items = len(items)

    for pid in items:
        #идем в цикле по товарам
        idx.append(pid) #добавляем товар в список
        #values.append(1.0 / n_items) #нормализуем 
        #values.append(1.0)
        values.append(emb_pur_count[d_user_id][pid])
    return sp.coo_matrix(
        (np.array(values).astype(np.float32), ([0] * len(idx), idx)), shape=(1, 49465),
    )

In [ ]:
#подготовим данные для обучения
train_rows = []
usr_id = 0
#идем в цикле по train_matrix
for i in tqdm(train_matrix):
  temp = np.nonzero(i)[0] #получаем id товаров 
  train_rows.append(make_coo_row(temp,usr_id)) #добавляем coo_matrix в список train_rows
  usr_id += 1 #инкрементим пользователя

In [ ]:
#сложим матрицу по столбцам и сожмем
matrix_train_sparse = sp.vstack(train_rows).tocsr()


In [ ]:
matrix_train_sparse.shape

(100000, 49465)

In [ ]:
#создадим модель ALS
model_als = implicit.als.AlternatingLeastSquares(factors=128, regularization=0.5, iterations=7,use_gpu=False)
#обучим модель ALS
#транспонируем матрицу так как модель принимает (number_of_items, number_of_users).
model_als.fit(matrix_train_sparse.T)

In [ ]:
#создадим датасет рекомендуемых товаров
df_result_als = pd.DataFrame(columns=['user_id','product_id'])

In [ ]:
###ALS
m_ap_als = [] #список для сохранения матрике по пользователю
usr_id = 0
#Цикл по train_matrix, далее подготавливаем данные и получаем прогноз рекомендации
for index,values in tqdm(enumerate(train_matrix)):
  #подготовим данные используя функцию make_coo_row
  row_sparse = make_coo_row(np.nonzero(values)[0],usr_id).tocsr()
  #получим прогноз рекомендации
  raw_recs = model_als.recommend(0, row_sparse, N=10, filter_already_liked_items=False, recalculate_user=True)
  #докодируем товар полученный из рекомендации
  recommended_items = [encode_product_id.get(i[0]) for i in raw_recs]
  #получим товар который купил пользователь в будущем
  tmp = np.nonzero(test_matrix[index])[0]
  #декодируем товар который купил пользователь в будущем
  gt_items = [encode_product_id.get(i) for i in tmp] 
  #посчитаем метрику
  ap = normalized_average_precision(gt_items, np.array(recommended_items))
  #добавим список рекомендаций в датасет
  df_result_als = df_result_als.append({'user_id':encode_user_id.get(index),'product_id':recommended_items},ignore_index=True)
  #добавим метрику в список
  m_ap_als.append(ap)
  #инкрементим пользователя
  usr_id += 1
print(np.mean(m_ap_als))


0.29388479407438906


In [ ]:
#создадим модель ближайших соседей для item to item
model_nearest_neighbours = implicit.nearest_neighbours.CosineRecommender(K=200)
model_nearest_neighbours.fit(matrix_train_sparse.T)

/usr/local/lib/python3.7/dist-packages/implicit/nearest_neighbours.py:167: RuntimeWarning: invalid value encountered in true_divide
  X.data = X.data / sqrt(bincount(X.row, X.data ** 2))[X.row]


In [ ]:
#создадим датасет рекомендуемых товаров
df_result_item_knn = pd.DataFrame(columns=['user_id','product_id'])

In [ ]:
###item to teim NN
m_ap_nn = [] #список для сохранения матрике по пользователю
usr_id = 0
#Цикл по train_matrix, далее подготавливаем данные и получаем прогноз рекомендации
for index,values in tqdm(enumerate(train_matrix)):
  #подготовим данные используя функцию make_coo_row
  row_sparse = make_coo_row(np.nonzero(values)[0],usr_id).tocsr()
  #получим прогноз рекомендации
  raw_recs = model_nearest_neighbours.recommend(0, row_sparse, N=10, filter_already_liked_items=False, recalculate_user=True)
  #докодируем товар полученный из рекомендации
  recommended_items = [encode_product_id.get(i[0]) for i in raw_recs]
  #получим товар который купил пользователь в будущем
  tmp = np.nonzero(test_matrix[index])[0]
  #декодируем товар который купил пользователь в будущем
  gt_items = [encode_product_id.get(i) for i in tmp] 
  #посчитаем метрику
  ap = normalized_average_precision(gt_items, np.array(recommended_items))
  #добавим список рекомендаций в датасет
  df_result_item_knn = df_result_item_knn.append({'user_id':encode_user_id.get(index),'product_id':recommended_items},ignore_index=True)
  #добавим метрику в список
  m_ap_nn.append(ap)
  #инкрементим пользователя
  usr_id += 1
print(np.mean(m_ap_nn))


0.32210353518833457


In [ ]:
#user_2_user KNN

In [ ]:
#сложим матрицу по столбцам
X_sparse = sp.vstack(train_rows)
#сожмем матрицу
X_stored = X_sparse.tocsr()

In [ ]:
#снизим размероность используя SVD
svd = TruncatedSVD(n_components=128)
X_dense = svd.fit_transform(X_sparse)

In [ ]:
#зададим количество соседей
num_neighbours = 256
#создадим модель
knn = NearestNeighbors(n_neighbors=num_neighbours, metric="cosine")
#обучим модель
knn.fit(X_dense)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=256, p=2,
                 radius=1.0)

In [ ]:
#создадим датасет рекомендуемых товаров
df_result_user_knn = pd.DataFrame(columns=['user_id','product_id'])

In [ ]:
####user_2_user KNN
m_ap_user_knn = []
usr_id = 0
for index,values in tqdm(enumerate(train_matrix)):
  #подготовим данные для подачи в модель
  row_sparse = make_coo_row(np.nonzero(values)[0],usr_id)
  #понизим размерность
  row_dense = svd.transform(row_sparse)
  #получим прогноз
  knn_result = knn.kneighbors(row_dense, n_neighbors=num_neighbours)
  #получим ближайших соседий (n_queries, n_neighbors)
  neighbors = knn_result[1]
  #получим сумму и переведем в массив numpy и свернем массив в одно измерение
  scores = np.asarray(X_stored[neighbors[0]].sum(axis=0)[0]).flatten()
  #получим индексы отсортированного массива
  top_indices = np.argsort(-scores)
  #декодируем полученные рекомендации
  recommended_items = [encode_product_id.get(i) for i in top_indices[:10]]
  #получим товары которые покупал пользователь
  tmp = np.nonzero(test_matrix[index])[0]
  #декодируем товары которые покупал пользователь
  gt_items = [encode_product_id.get(i) for i in tmp] 
  #посчитаем метрику
  ap = normalized_average_precision(gt_items, np.array(recommended_items))
  #добавим результат в датасет
  df_result_user_knn = df_result_user_knn.append({'user_id':encode_user_id.get(index),'product_id':recommended_items},ignore_index=True)
  usr_id += 1
  m_ap_user_knn.append(ap)
print(np.mean(m_ap_user_knn))


0.31127223622921385


In [ ]:
#подготовим датасет для kaggle

In [ ]:
def conv(x):
  g = str()
  for i in (x):
    g += str(i) + ' '
  return g

In [ ]:
f = lambda x: conv(x['product_id']) 

In [ ]:
df_result_als['test'] = df_result_als.apply(f,axis=1)
df_result_item_knn['test'] = df_result_item_knn.apply(f,axis=1)
df_result_user_knn['test'] = df_result_user_knn.apply(f,axis=1)

In [ ]:
df_result_als.drop(columns=['product_id'],inplace=True)
df_result_item_knn.drop(columns=['product_id'],inplace=True)
df_result_user_knn.drop(columns=['product_id'],inplace=True)

In [ ]:
df_result_als.rename(columns={"test": "product_id"},inplace=True)
df_result_item_knn.rename(columns={"test": "product_id"},inplace=True)
df_result_user_knn.rename(columns={"test": "product_id"},inplace=True)

In [ ]:
df_result_als.to_csv('/content/drive/MyDrive/skillbox-recommender-system.zip (Unzipped Files)/df_result_als.csv',index=False)
df_result_item_knn.to_csv('/content/drive/MyDrive/skillbox-recommender-system.zip (Unzipped Files)/df_result_item_knn.csv',index=False)
df_result_user_knn.to_csv('/content/drive/MyDrive/skillbox-recommender-system.zip (Unzipped Files)/df_result_user_knn.csv',index=False)